# Time Series Analysis of Open Power Systems Data

We will utilize the time series tools on the Pandas library to organize, transform, analyze and visualize our data in this analysis. We will focus our attention on the wrangling and visualization aspects.

## The Data

Our data for this analysis is [Germany's Open Power Systems Data (OPSD)](https://github.com/jenfly/opsd/raw/master/opsd_germany_daily.csv), which has daily observations. We know that Germany has been growing their renewable energy production for the past few years, and this data set contains the country's total electicity consumption, wind and solar power production form 2006 to 2017.

The energy production and consumption records are in units of gigawatt-hours (GWh). The data dictionary is as follows:

+ Date: date in yyyy-mm-dd format
+ Consumption: energy consumption in GWh
+ Wind: wind energy production in GWh
+ Solar: solar energy production in GWh
+ Wind+Solar: Sum of the wind and solar energy production in GWh

We will use this data to investigate how electricity consumption and production in Germany has varied over time. We are attempting to answer the following questions:

+ When is energy consumption typically highest?
+ What, if any, are the seasonality affects throughout a year for energy production and consumption?
+ What are the long-term trends in energy consumption as well as in wind and solar energy production?
+ How does the prodcution of wind and solar energy compare to the country's energy consumption, and what has the production-to-consumption ratio changed over the years?

In [1]:
# Import libraries/packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Read in the data as a dataframe
opsd_daily = pd.read_csv('data/opsd_germany_daily.csv')

# Checl the shape of the dataframe
opsd_daily.shape

(4383, 5)

In [3]:
# Inspect head of dataframe
opsd_daily.head(3)

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN


In [4]:
# Inspect tail of dataframe
opsd_daily.tail(3)

,Date,Consumption,Wind,Solar,Wind+Solar
4380,2017-12-29,1295.08753,584.277,29.854,614.131
4381,2017-12-30,1215.44897,721.247,7.467,728.714
4382,2017-12-31,1107.11488,721.176,19.980,741.156


The data starts January 1, 2006 and ends December 31, 2017 and is spread over 4383 observations.

In [5]:
# Inspect data types
opsd_daily.dtypes

Date            object
Consumption    float64
Wind           float64
Solar          float64
Wind+Solar     float64
dtype: object

We would like to use the Date field as the index for this time series, we it is preferable to have it in DatetimeIndex type. It is object type currently (Pandas is treating it as text).

In [6]:
# Convert date field to datetime type
opsd_daily['Date'] = pd.to_datetime(opsd_daily['Date'])

# Inspect data types to validate it was converted
opsd_daily.dtypes

Date           datetime64[ns]
Consumption           float64
Wind                  float64
Solar                 float64
Wind+Solar            float64
dtype: object

In [7]:
# Set Date field as the index
opsd_daily = opsd_daily.set_index('Date')

# Inspect the dataframe head
opsd_daily.head(3)

,Consumption,Wind,Solar,Wind+Solar
Date,,,,
2006-01-01,1069.184,NaN,NaN,NaN
2006-01-02,1380.521,NaN,NaN,NaN
2006-01-03,1442.533,NaN,NaN,NaN


In [8]:
# Inspect the datetime index
opsd_daily.index

DatetimeIndex(['2006-01-01', '2006-01-02', '2006-01-03', '2006-01-04',
               '2006-01-05', '2006-01-06', '2006-01-07', '2006-01-08',
               '2006-01-09', '2006-01-10',
               ...
               '2017-12-22', '2017-12-23', '2017-12-24', '2017-12-25',
               '2017-12-26', '2017-12-27', '2017-12-28', '2017-12-29',
               '2017-12-30', '2017-12-31'],
              dtype='datetime64[ns]', name='Date', length=4383, freq=None)

The previous lines may be more efficiently done in a single line whil reading in the data, as shown in the cell below.

In [12]:
# Read in the data as a time series, with date as the index and in Datetime type
#opsd_daily = pd.read_csv('data/opsd_germany_daily.csv', index_col=0, parse_dates=True)

The with the dates in theis Datetime format, Pandas allows for the day, month, and year to be accessed as attributes. This enables us to be flexible in the wrangling, anlaysis, and modifying of our dataframe.

In [15]:
# Add year, month, and days to the dataframe
opsd_daily['Year'] = opsd_daily.index.year
opsd_daily['Month'] = opsd_daily.index.month
opsd_daily['Weekday Name'] = opsd_daily.index.weekday_name

# Inspect a random sample of 5 observations
opsd_daily.sample(5)

,Consumption,Wind,Solar,Wind+Solar,Year,Month,Weekday Name
Date,,,,,,,
2009-05-24,932.701,NaN,NaN,NaN,2009,5,Sunday
2014-10-16,1480.270,61.472,40.676,102.148,2014,10,Thursday
2016-06-06,1393.733,69.562,217.347,286.909,2016,6,Monday
2013-06-09,971.738,44.830,135.025,179.855,2013,6,Sunday
2010-11-07,1151.835,28.397,NaN,NaN,2010,11,Sunday


Another useful feature Pandas allows us with this format is the ability to access data with .loc, so that we may grab single records, or slices, and even partial matches.

In [16]:
# Inspect the record for August 23, 2008
opsd_daily.loc['2008-08-23']

Consumption      1152.01
Wind                 NaN
Solar                NaN
Wind+Solar           NaN
Year                2008
Month                  8
Weekday Name    Saturday
Name: 2008-08-23 00:00:00, dtype: object

In [18]:
# Inspect the days of October 24 and 25, 2017
opsd_daily.loc['2017-10-24' : '2017-10-25']

,Consumption,Wind,Solar,Wind+Solar,Year,Month,Weekday Name
Date,,,,,,,
2017-10-24,1508.51484,290.400,41.479,331.879,2017,10,Tuesday
2017-10-25,1506.14869,454.959,52.361,507.320,2017,10,Wednesday


In [19]:
# Inspect the month of June, 2014
opsd_daily.loc['2014-06']

,Consumption,Wind,Solar,Wind+Solar,Year,Month,Weekday Name
Date,,,,,,,
2014-06-01,1068.982,23.571,155.656,179.227,2014,6,Sunday
2014-06-02,1403.083,15.254,169.436,184.690,2014,6,Monday
2014-06-03,1433.509,14.213,166.671,180.884,2014,6,Tuesday
2014-06-04,1444.044,67.454,153.844,221.298,2014,6,Wednesday
2014-06-05,1436.595,176.676,158.335,335.011,2014,6,Thursday
2014-06-06,1403.466,61.768,212.446,274.214,2014,6,Friday
2014-06-07,1176.898,38.270,203.093,241.363,2014,6,Saturday
2014-06-08,1029.637,49.895,192.656,242.551,2014,6,Sunday
2014-06-09,1075.393,76.425,195.666,272.091,2014,6,Monday


## Visualizing The Time Series